<a href="https://colab.research.google.com/github/Fsrkh/Persian-dates-NLP/blob/main/persian_dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
from hazm import *
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense, BatchNormalization
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight # Import the necessary function
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
import re

Import data

In [73]:
data = pd.read_csv('dates.csv')
data.head()

,Formal Date,Informal Date 1,Informal Date 2,Informal Date 3,Informal Date 4,Informal Date 5,Informal Date 6,Informal Date 7,Informal Date 8,Informal Date 9
0,1400/01/01,اول فروردین هزار و چهارصد,یک فروردین هزار و چهارصد,یک فروردین ۱۴۰۰,فروردین یک ۱۴۰۰,اول فروردین ۱۴۰۰,یکِ فروردین ۱۴۰۰,یکم فروردین ۱۴۰۰,اول فروردین ماه ۱۴۰۰,فروردینِ یک ۱۴۰۰
1,1400/02/10,دهم اردیبهشت هزار و چهارصد,ده اردیبهشت هزار و چهارصد,ده اردیبهشت ۱۴۰۰,اردیبهشت ده ۱۴۰۰,دهم اردیبهشت ۱۴۰۰,دهِ اردیبهشت ۱۴۰۰,دهم اردیبهشت ماه ۱۴۰۰,اردیبهشتِ ده ۱۴۰۰,ده اردیبهشت ۱۴۰۰
2,1401/03/05,پنجم خرداد هزار و چهارصد و یک,پنج خرداد هزار و چهارصد و یک,پنج خرداد ۱۴۰۱,خرداد پنج ۱۴۰۱,پنجم خرداد ۱۴۰۱,پنجِ خرداد ۱۴۰۱,پنجم خرداد ماه ۱۴۰۱,خردادِ پنج ۱۴۰۱,پنج خرداد ۱۴۰۱
3,1401/04/15,پانزدهم تیر هزار و چهارصد و یک,پانزده تیر هزار و چهارصد و یک,پانزده تیر ۱۴۰۱,تیر پانزده ۱۴۰۱,پانزدهم تیر ۱۴۰۱,پانزدهِ تیر ۱۴۰۱,پانزدهم تیر ماه ۱۴۰۱,تیرِ پانزده ۱۴۰۱,پانزده تیر ۱۴۰۱
4,1402/05/20,بیستم مرداد هزار و چهارصد و دو,بیست مرداد هزار و چهارصد و دو,بیست مرداد ۱۴۰۲,مرداد بیست ۱۴۰۲,بیستم مرداد ۱۴۰۲,بیستِ مرداد ۱۴۰۲,بیستم مرداد ماه ۱۴۰۲,مردادِ بیست ۱۴۰۲,بیست مرداد ۱۴۰۲


In [74]:
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values)

Missing values in each column:
 Formal Date          0
Informal Date 1      0
Informal Date 2      0
Informal Date 3      0
Informal Date 4      0
Informal Date 5      0
Informal Date 6    623
Informal Date 7    623
Informal Date 8    623
Informal Date 9    623
dtype: int64


In [75]:
data['Informal Dates'] = data.iloc[:, 1:].apply(lambda x: ' '.join(x.dropna()), axis=1)

y = data['Formal Date']

X = data['Informal Dates']

In [76]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text

In [77]:
data['Informal Dates'] = data['Informal Dates'].apply(preprocess_text)

In [78]:
y.shape

(761,)

In [79]:
X.shape

(761,)

#Tokenazation and Encoding

In [80]:
hazm_tokenizer = WordTokenizer(replace_numbers=False, replace_ids=True)
X = X.apply(lambda v: hazm_tokenizer.tokenize(v))
X.head()

,Informal Dates
0,"[اول, فروردین, هزار, و, چهارصد, یک, فروردین, ه..."
1,"[دهم, اردیبهشت, هزار, و, چهارصد, ده, اردیبهشت,..."
2,"[پنجم, خرداد, هزار, و, چهارصد, و, یک, پنج, خرد..."
3,"[پانزدهم, تیر, هزار, و, چهارصد, و, یک, پانزده,..."
4,"[بیستم, مرداد, هزار, و, چهارصد, و, دو, بیست, م..."


In [81]:
hazm_lemmatizer = Lemmatizer()
X = X.apply(lambda v: [hazm_lemmatizer.lemmatize(w) for w in v])
X.head()

,Informal Dates
0,"[اول, فروردین, هزار, و, چهارصد, یک, فروردین, ه..."
1,"[دهم, اردیبهشت, هزار, و, چهارصد, ده, اردیبهشت,..."
2,"[پنجم, خرداد, هزار, و, چهارصد, و, یک, پنج, خرد..."
3,"[پانزدهم, تیر, هزار, و, چهارصد, و, یک, پانزده,..."
4,"[بیستم, مرداد, هزار, و, چهارصد, و, دو, بیست, م..."


In [84]:
X = X.apply(lambda v: ' '.join(v))
X = list(X)
X[:5]

['فروردین هزار چهارصد فروردین هزار چهارصد فروردین ۱۴۰۰ فروردین ۱۴۰۰ فروردین ۱۴۰۰ یکِ فروردین ۱۴۰۰ یکم فروردین ۱۴۰۰ فروردین ماه ۱۴۰۰ فروردینِ ۱۴۰۰',
 'دهم اردیبهشت هزار چهارصد ده اردیبهشت هزار چهارصد ده اردیبهشت ۱۴۰۰ اردیبهشت ده ۱۴۰۰ دهم اردیبهشت ۱۴۰۰ دهِ اردیبهشت ۱۴۰۰ دهم اردیبهشت ماه ۱۴۰۰ اردیبهشتِ ده ۱۴۰۰ ده اردیبهشت ۱۴۰۰',
 'پنجم خرداد هزار چهارصد خرداد هزار چهارصد خرداد ۱۴۰۱ خرداد ۱۴۰۱ پنجم خرداد ۱۴۰۱ پنجِ خرداد ۱۴۰۱ پنجم خرداد ماه ۱۴۰۱ خردادِ ۱۴۰۱ خرداد ۱۴۰۱',
 'پانزدهم تیر هزار چهارصد پانزده تیر هزار چهارصد پانزده تیر ۱۴۰۱ تیر پانزده ۱۴۰۱ پانزدهم تیر ۱۴۰۱ پانزدهِ تیر ۱۴۰۱ پانزدهم تیر ماه ۱۴۰۱ تیرِ پانزده ۱۴۰۱ پانزده تیر ۱۴۰۱',
 'بیستم مرداد هزار چهارصد بیست مرداد هزار چهارصد بیست مرداد ۱۴۰۲ مرداد بیست ۱۴۰۲ بیستم مرداد ۱۴۰۲ بیستِ مرداد ۱۴۰۲ بیستم مرداد ماه ۱۴۰۲ مردادِ بیست ۱۴۰۲ بیست مرداد ۱۴۰۲']

In [85]:
max_length = max(len(seq) for seq in X_sequences)
X_padded = pad_sequences(X_sequences, maxlen=max_length, padding='post')
X_padded

NameError: name 'X_sequences' is not defined

In [86]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded

array([487, 488, 493, 495, 497, 502, 508, 509, 513, 514, 517, 521, 522,
       523, 524, 526, 527, 528, 530, 531, 536, 537, 539, 541, 542, 543,
       544, 547, 548, 549, 556, 557, 561, 562, 565, 567, 568, 569, 570,
       571, 573, 574, 578, 579, 583, 584, 585, 587, 207, 209, 215, 218,
       220, 222, 226, 230, 233, 236, 240, 242, 243, 245, 246, 248, 250,
       252, 254, 258, 261, 269, 272, 275, 276, 279, 283, 287, 290, 293,
       297, 299, 302, 305, 308, 311, 312, 313, 315, 317, 319, 324, 327,
       329, 333, 337, 342, 346, 347, 353, 356, 359, 360, 364, 367, 368,
       372, 375, 378, 380, 381, 383, 387, 389, 394, 398, 402, 406, 413,
       417, 420, 422, 424, 426, 428, 433, 438, 439, 440, 444,   0,  41,
        56,  82,  91, 108, 127, 158, 169, 184,  20,  42,  52, 107, 168,
       205, 212, 362, 490, 552, 589, 620, 637, 675, 700, 727, 738, 216,
       267, 304, 334, 371, 404, 445, 461, 506, 555, 576, 609, 632, 641,
       681, 715, 730,   2,  44,  69, 115, 146, 186, 244, 307, 37

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [88]:
X_train

['بیستم بهمن هزار سیصد شصت بیست بهمن ۱۳۶۶ بهمن بیست ۱۳۶۶ بیستم بهمن ۱۳۶۶ بهمن بیست ۱۳۶۶',
 'بیست نهم تیر هزار سیصد نود بیست تیر ۱۳۹۱ تیر بیست ۱۳۹۱ بیست نهم تیر ۱۳۹۱ تیر بیست ۱۳۹۱',
 'هفدهم بهمن هزار سیصد بیست هشت هفده بهمن ۱۳۲۸ بهمن هفده ۱۳۲۸ هفدهم بهمن ۱۳۲۸ بهمن هفده ۱۳۲۸',
 'دهم تیر هزار چهارصد ده تیر ۱۴۰۱ تیر ده ۱۴۰۱ دهم تیر ۱۴۰۱ تیر ده ۱۴۰۱',
 'سی\u200cام تیر هزار سیصد شست هفت تیر ۱۳۶۷ تیر ۱۳۶۷ سی\u200cام تیر ۱۳۶۷ تیر ۱۳۶۷',
 'آذر هزار دویست آذر ۱۲۰۵ آذر ۱۲۰۵ آذر ۱۲۰۵ آذر ۱۲۰۵',
 'بیست اردیبهشت هزار سیصد هشتاد بیست اردیبهشت ۱۳۸۳ اردیبهشت بیست ۱۳۸۳ بیست اردیبهشت ۱۳۸۳ اردیبهشت بیست ۱۳۸۳',
 'پانزدهم بهمن هزار دویست پنجاه پانزده بهمن ۱۲۵۳ بهمن پانزده ۱۲۵۳ پانزدهم بهمن ۱۲۵۳ بهمن پانزده ۱۲۵۳',
 'بیستم بهمن هزار دویست پنجاه بیست بهمن ۱۲۵۵ بهمن بیست ۱۲۵۵ بیستم بهمن ۱۲۵۵ بهمن بیست ۱۲۵۵',
 'سی\u200cام خرداد هزار سیصد شصت خرداد ۱۳۶۶ خرداد ۱۳۶۶ سی\u200cام خرداد ۱۳۶۶ خرداد ۱۳۶۶',
 'پنجم فروردین هزار سیصد دوازده فروردین هزار سیصد دوازده فروردین ۱۳۱۲ فروردین ۱۳۱۲ پنجم فروردین ۱۳۱۲ پنجِ فروردین ۱۳

In [89]:
y_train

,Formal Date
160,1366/11/20
199,1391/04/29
278,1328/11/17
704,1401/04/10
728,1367/04/30
...,...
614,1448/10/20
430,1433/07/18
688,1347/12/24
486,1203/03/06


In [90]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

AttributeError: 'list' object has no attribute 'shape'

# Building and Training the model

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [91]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))

model.add(Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001))))
model.add(Dropout(0.4))

model.add(Bidirectional(LSTM(32, kernel_regularizer=l2(0.001))))
model.add(Dropout(0.4))

model.add(Dense(len(label_encoder.classes_), activation='softmax'))

AttributeError: 'WordTokenizer' object has no attribute 'word_index'

In [92]:
adam_optimizer = Adam()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=16,
                    class_weight=class_weights_dict, callbacks=[early_stopping, reduce_lr])

#Prediction

In [ ]:
def predict_formal_date(informal_date):
    seq = tokenizer.texts_to_sequences([preprocess_text(informal_date)])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    pred = model.predict(padded)
    return label_encoder.inverse_transform([np.argmax(pred)])

In [ ]:
new_informal_date = "دهم اردیبهشت هزار و چهارصد"
predicted_date = predict_formal_date(new_informal_date)
print(f"Predicted formal date: {predicted_date[0]}")